In [3]:
from app import app, db, bcrypt
from models import User, Stock, Portfolio
from flask import jsonify, request
import pandas as pd
import sqlite3

In [4]:
data = [
  {
    "date": "Sat, 01 Mar 2025 00:00:00 GMT",
    "id": 2,
    "purchase_price": 1600.0,
    "quantity": 10,
    "symbol": "HDFCBANK.NS"
  },
  {
    "date": "Sat, 01 Feb 2025 00:00:00 GMT",
    "id": 3,
    "purchase_price": 2200.0,
    "quantity": 20,
    "symbol": "HINDUNILVR.NS"
  },
  {
    "date": "Tue, 04 Mar 2025 00:00:00 GMT",
    "id": 4,
    "purchase_price": 3500.0,
    "quantity": 30,
    "symbol": "TCS.NS"
  },
  {
    "date": "Fri, 04 Aug 2023 00:00:00 GMT",
    "id": 5,
    "purchase_price": 900.0,
    "quantity": 10,
    "symbol": "BBTC.NS"
  },
  {
    "date": "Sat, 01 Feb 2025 00:00:00 GMT",
    "id": 6,
    "purchase_price": 1200.0,
    "quantity": 20,
    "symbol": "BALKRISIND.NS"
  },
  {
    "date": "Tue, 10 Sep 2019 00:00:00 GMT",
    "id": 12,
    "purchase_price": 1400.0,
    "quantity": 90,
    "symbol": "HDFCBANK.NS"
  },
  {
    "date": "Wed, 19 May 2021 00:00:00 GMT",
    "id": 13,
    "purchase_price": 1500.0,
    "quantity": 30,
    "symbol": "HDFCBANK.NS"
  },
  {
    "date": "Wed, 08 Feb 2023 00:00:00 GMT",
    "id": 14,
    "purchase_price": 1450.0,
    "quantity": 70,
    "symbol": "HDFCBANK.NS"
  },
  {
    "date": "Wed, 08 Feb 2023 00:00:00 GMT",
    "id": 15,
    "purchase_price": 2450.0,
    "quantity": 70,
    "symbol": "HINDUNILVR.NS"
  },
  {
    "date": "Thu, 11 Mar 2021 00:00:00 GMT",
    "id": 16,
    "purchase_price": 2150.0,
    "quantity": 90,
    "symbol": "HINDUNILVR.NS"
  },
  {
    "date": "Wed, 10 Jul 2019 00:00:00 GMT",
    "id": 17,
    "purchase_price": 3000.0,
    "quantity": 60,
    "symbol": "TCS.NS"
  },
  {
    "date": "Tue, 11 Mar 2025 00:00:00 GMT",
    "id": 18,
    "purchase_price": 1400.0,
    "quantity": 40,
    "symbol": "BALKRISIND.NS"
  },
  {
    "date": "Tue, 11 Mar 2025 00:00:00 GMT",
    "id": 19,
    "purchase_price": 1400.0,
    "quantity": 40,
    "symbol": "BALKRISIND.NS"
  }
]


In [36]:
import sys
import json
import sqlite3
from datetime import datetime, date, timedelta

#data_input = sys.argv[1]

# start and end date
startDate = date.today() - timedelta(days=365)
endDate = date.today()

# Factory Pattern for Trade Creation
class Trade:
    def __init__(self, id, symbol, purchase_price, quantity, date):
        self.id = id
        self.symbol = symbol
        self.purchase_price = purchase_price
        self.quantity = quantity
        self.date = datetime.strptime(date, '%a, %d %b %Y %H:%M:%S %Z')

    def __str__(self):
        return f"Trade(symbol={self.symbol}, purchase_price={self.purchase_price})"

    def lastPrice(self):
        return PortfolioValueProvider().get_asset_value(self.symbol,"Close", startDate, endDate)

    def totalValueNew(self):
        return self.quantity * self.lastPrice()


class TradeFactory:
    def create_trade(self, id, symbol, purchase_price, quantity, date):
        return Trade(id, symbol, purchase_price, quantity, date)


# Portfolio Class with Aggregation and API Interaction
class Portfolio:
    def __init__(self):
        self.trades = []
        self.value_provider = PortfolioValueProvider()
        self.net_values = None  # Store the calculated net values
        self.portfolio_status = {}  # Store the portfolio's status for each date

    def add_trade(self, trade):
        self.trades.append(trade)
        self.update_portfolio_status()

    def add_trades(self, trade_data):
        trade_factory = TradeFactory()
        for trade_info in trade_data:
            self.trades.append(trade_factory.create_trade(**trade_info))
        self.update_portfolio_status()

    def calculate_net(self):
        if self.net_values is None:
            net_value = {}
            total_quantity = {}
            average_price = {}
            current_value = {}
            for trade in self.trades:
                if trade.symbol in net_value:
                    net_value[trade.symbol] += trade.quantity * trade.purchase_price
                    total_quantity[trade.symbol] += trade.quantity
                    average_price[trade.symbol] = net_value[trade.symbol] / total_quantity[trade.symbol]
                    current_value[trade.symbol] = total_quantity[trade.symbol] * self.value_provider.get_asset_value(
                        trade.symbol, "Close", startDate, endDate)
                else:
                    net_value[trade.symbol] = trade.quantity * trade.purchase_price
                    total_quantity[trade.symbol] = trade.quantity
                    average_price[trade.symbol] = net_value[trade.symbol] / total_quantity[trade.symbol]
                    current_value[trade.symbol] = total_quantity[trade.symbol] * self.value_provider.get_asset_value(
                        trade.symbol, "Close", startDate, endDate)
            self.net_values = net_value, total_quantity, average_price, current_value

        return self.net_values

    def calculate_current_value(self, currency):
        current_value = {}
        for trade in self.trades:
            currency_converter = 1
            asset_value = self.value_provider.get_asset_value(trade.symbol, "Close", startDate, trade.date)
            if asset_value is not None:
                current_value[trade.symbol] = asset_value * trade.quantity * currency_converter
        return current_value

    def update_portfolio_status(self):
        for trade in self.trades:
            if trade.date not in self.portfolio_status:
                self.portfolio_status[trade.date] = {}

            if trade.symbol not in self.portfolio_status[trade.date]:
                self.portfolio_status[trade.date][trade.symbol] = trade.quantity
            else:
                self.portfolio_status[trade.date][trade.symbol] += trade.quantity

    def get_portfolio_status_by_date(self, date):
        return self.portfolio_status.get(date, {})

    def get_portfolio_status_for_dates(self, dates):
        portfolio_status_for_dates = []

        for date in dates:
            portfolio_status_for_dates.append({
                'date': date,
                'assets_quantity': self.get_portfolio_status_by_date(date)
            })

        return portfolio_status_for_dates

    def calculate_daily_values_oneyear(self, currency):
        daily_values = []
        trade_dates = []
        stock_quantity = {}
        start_dt = startDate
        end_dt = endDate
        delta = timedelta(days=1)

        # store the dates between two dates in a list
        dates_oneyear = []

        while start_dt <= end_dt:
            dates_oneyear.append(start_dt.isoformat())
            start_dt += delta

        sorted_trades = sorted(self.trades, key=lambda trade: trade.date)
        for date in dates_oneyear:
            total_value_for_date = 0
            for trade in sorted_trades:
                if trade.date == date:
                    total_value_for_date += trade.quantity * self.value_provider.get_asset_value(trade.symbol, date)
            daily_values.append(total_value_for_date)

        portfolio_status = self.get_portfolio_status_for_dates(dates_oneyear)

        return portfolio_status, dates_oneyear

    def assets_on_dates_oneyear(self):
        dates_oneyear = []
        start_dt = startDate
        end_dt = endDate
        delta = timedelta(days=1)

        while start_dt <= end_dt:
            dates_oneyear.append(start_dt.isoformat())
            start_dt += delta

        assets_by_date = {}

        trade_dates_set = set(trade.date for trade in self.trades)
        sorted_trades = sorted(self.trades, key=lambda trade: trade.date)

        for date1 in dates_oneyear:
            assets_on_date = {}
            date1_dt = datetime.fromisoformat(date1)  # Convert date1 to datetime object
            for trade in self.trades:
                if trade.date <= date1_dt:
                    if trade.symbol in assets_on_date:
                        assets_on_date[trade.symbol] += trade.quantity
                    else:
                        assets_on_date[trade.symbol] = trade.quantity
            assets_by_date[date1] = assets_on_date

        return assets_by_date

    @staticmethod
    def portfolio_value_for_dates(portfolio_dict, currency):
        stock_data = {}
        value_provider = PortfolioValueProvider()
        #startDate = date.today() - timedelta(days=365)
        #endDate = date.today()

        for date_str, stocks in portfolio_dict.items():
            date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()  # Convert date string to date object
            for stock_name in stocks.keys():
                if stock_name not in stock_data:
                    stock_data[stock_name] = value_provider.get_asset_values(stock_name, "Close", startDate, endDate)

        portfolio_value_by_dates = {}

        for date_str, stocks in portfolio_dict.items():
            # Convert date string to datetime object with time set to midnight
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
            date_key = date_obj.strftime('%Y-%m-%d %H:%M:%S')  # Format to match stock_history keys
            total_value = 0

            for stock_name, quantity in stocks.items():
                stock_history = stock_data[stock_name]

                if date_key in stock_history:
                    stock_value = stock_history[date_key] * quantity
                    total_value += stock_value

            if total_value > 0:
                portfolio_value_by_dates[date_str] = total_value
        return portfolio_value_by_dates



    """
    def portfolio_value_for_dates(portfolio_dict, currency):
        stock_data = {}

        for date, stocks in portfolio_dict.items():
            for stock_name in stocks.keys():
                if stock_name not in stock_data:
                    stock_data[stock_name] = PortfolioValueProvider().get_asset_values(stock_name,"Close", startDate, endDate)

        portfolio_value_by_dates = {}

        for date, stocks in portfolio_dict.items():
            total_value = 0

            for stock_name, quantity in stocks.items():
                stock_history = stock_data[stock_name]

                if date in stock_history:
                    stock_value = stock_history[date] * quantity
                    total_value += stock_value

            if total_value > 0:
                portfolio_value_by_dates[date] = total_value
        return portfolio_value_by_dates
    """



# Data Collection
class PortfolioValueProvider:
    def __init__(self):
        self.conn = sqlite3.connect('stock_data.db')
        self.cursor = self.conn.cursor()


    def get_asset_value(self, symbol, metric, startDate, endDate):
        query = """
        SELECT d.value
        FROM Data d
        JOIN Companies c ON d.Company_ID = c.Company_ID
        JOIN Metrics m ON d.Metric_ID = m.Metric_ID
        WHERE c.company = ?
        AND m.metric = ?
        AND d.Date BETWEEN ? AND ?
        ORDER BY date DESC LIMIT 1
        """
        self.cursor.execute(query, (symbol, metric, startDate, endDate))
        result = self.cursor.fetchone()
        return result[0] if result else None

    def get_asset_values(self, symbol, metric, startDate, endDate):
        query = """
        SELECT d.Date, d.value
        FROM Data d
        JOIN Companies c ON d.Company_ID = c.Company_ID
        JOIN Metrics m ON d.Metric_ID = m.Metric_ID
        WHERE c.company = ?
        AND m.metric = ?
        AND d.Date BETWEEN ? AND ?
        ORDER BY d.date
        """
        self.cursor.execute(query, (symbol, metric, startDate, endDate))
        result = self.cursor.fetchall()
        return {row[0]: row[1] for row in result}  # Return a dictionary with dates as keys
        #return result
    #{row[0]: row[1] for row in result}

    @staticmethod
    def get_exchange_rate(country, target_currency='USD'):
        if country in PortfolioValueProvider.exchange_rates and target_currency in PortfolioValueProvider.exchange_rates[
            country]:
            return PortfolioValueProvider.exchange_rates[country][target_currency]
        else:
            return 1.0

    # Sample exchange rates (USD to other currencies)
    exchange_rates = {
        'USA': {
            'USD': 1.0,  # 1 USD to USD
            'INR': 75.0,  # Sample exchange rate: 1 USD to 75 INR
            'EUR': 0.85,  # Sample exchange rate: 1 USD to 0.85 EUR
        },
        'India': {
            'INR': 1.0,  # 1 INR to INR
            'USD': 0.013,  # Sample exchange rate: 1 INR to 0.013 USD
            'EUR': 0.011,  # Sample exchange rate: 1 INR to 0.011 EUR
        }
    }


# Output
class PortfolioOutput:
    def __init__(self, portfolio, currency):
        self.portfolio = portfolio
        self.currency = currency

    def portfolio_daily_value(self):
        return self.portfolio.calculate_current_value(self.currency)

    def portfolio_net_cost(self):
        return self.portfolio.calculate_net()[0]

    def portfolio_total_quantity(self):
        return self.portfolio.calculate_net()[1]

    def portfolio_net_cost_per_unit(self):
        return self.portfolio.calculate_net()[2]

    def portfolio_net(self):
        return self.portfolio.calculate_net()

    def portfolio_daily_values_oneyear(self):
        return self.portfolio.assets_on_dates_oneyear()

    def portfolio_value_date_rs(self):
        input_dict = self.portfolio_daily_values_oneyear()
        values = self.portfolio.portfolio_value_for_dates(input_dict, "USD")
        return values


# Final Output
class OutputForPHP:
    def __init__(self, currency):
        self.output_list = []
        self.portfolio = Portfolio()
        self.currency = currency

    def output_portfolio(self, trade_data):
        self.portfolio.add_trades(trade_data)
        portfolioOutput = PortfolioOutput(self.portfolio, self.currency)
        output = portfolioOutput.portfolio_net()
        output_currentValue = portfolioOutput.portfolio_daily_value()
        output_dailyvalues2 = portfolioOutput.portfolio_value_date_rs()
        output_dailyvalues = portfolioOutput.portfolio_daily_values_oneyear()

        output_list = [
            [
                key,
                [list(output[1].values())[i], list(output[0].values())[i], list(output[2].values())[i],
                 list(output[3].values())[i]]
            ]
            for i, key in enumerate(output[0].keys())
        ]
        return list(output_dailyvalues2.keys()), list(output_dailyvalues2.values()), output_list, list(output[0].values()), list(output[0].keys()), list(output[3].values())




In [37]:
def main(dataInput):
    #dataInput = json.loads(data_input)
    output = OutputForPHP("INR").output_portfolio(dataInput)
    # portfolio
    data_portfolio = output[2]

    # portfolio price chart data
    date_array = output[0]
    price_array = output[1]
    # final output array
    data_price = date_array, price_array, dataInput, data_portfolio, output[3], output[4], output[5]
    # Date Array, Price Array, Input data, Portfolio Array

    return data_price

In [38]:
main(data)

(['2024-03-26',
  '2024-03-27',
  '2024-03-28',
  '2024-04-01',
  '2024-04-02',
  '2024-04-03',
  '2024-04-04',
  '2024-04-05',
  '2024-04-08',
  '2024-04-09',
  '2024-04-10',
  '2024-04-12',
  '2024-04-15',
  '2024-04-16',
  '2024-04-18',
  '2024-04-19',
  '2024-04-22',
  '2024-04-23',
  '2024-04-24',
  '2024-04-25',
  '2024-04-26',
  '2024-04-29',
  '2024-04-30',
  '2024-05-02',
  '2024-05-03',
  '2024-05-06',
  '2024-05-07',
  '2024-05-08',
  '2024-05-09',
  '2024-05-10',
  '2024-05-13',
  '2024-05-14',
  '2024-05-15',
  '2024-05-16',
  '2024-05-17',
  '2024-05-21',
  '2024-05-22',
  '2024-05-23',
  '2024-05-24',
  '2024-05-27',
  '2024-05-28',
  '2024-05-29',
  '2024-05-30',
  '2024-05-31',
  '2024-06-03',
  '2024-06-04',
  '2024-06-05',
  '2024-06-06',
  '2024-06-07',
  '2024-06-10',
  '2024-06-11',
  '2024-06-12',
  '2024-06-13',
  '2024-06-14',
  '2024-06-18',
  '2024-06-19',
  '2024-06-20',
  '2024-06-21',
  '2024-06-24',
  '2024-06-25',
  '2024-06-26',
  '2024-06-27',
  '2024-

In [8]:
Portfolio

__main__.Portfolio

In [9]:
data

[{'date': 'Sat, 01 Mar 2025 00:00:00 GMT',
  'id': 2,
  'purchase_price': 1600.0,
  'quantity': 10,
  'symbol': 'HDFCBANK.NS'},
 {'date': 'Sat, 01 Feb 2025 00:00:00 GMT',
  'id': 3,
  'purchase_price': 2200.0,
  'quantity': 20,
  'symbol': 'HINDUNILVR.NS'},
 {'date': 'Tue, 04 Mar 2025 00:00:00 GMT',
  'id': 4,
  'purchase_price': 3500.0,
  'quantity': 30,
  'symbol': 'TCS.NS'},
 {'date': 'Fri, 04 Aug 2023 00:00:00 GMT',
  'id': 5,
  'purchase_price': 900.0,
  'quantity': 10,
  'symbol': 'BBTC.NS'},
 {'date': 'Sat, 01 Feb 2025 00:00:00 GMT',
  'id': 6,
  'purchase_price': 1200.0,
  'quantity': 20,
  'symbol': 'BALKRISIND.NS'},
 {'date': 'Tue, 10 Sep 2019 00:00:00 GMT',
  'id': 12,
  'purchase_price': 1400.0,
  'quantity': 90,
  'symbol': 'HDFCBANK.NS'},
 {'date': 'Wed, 19 May 2021 00:00:00 GMT',
  'id': 13,
  'purchase_price': 1500.0,
  'quantity': 30,
  'symbol': 'HDFCBANK.NS'},
 {'date': 'Wed, 08 Feb 2023 00:00:00 GMT',
  'id': 14,
  'purchase_price': 1450.0,
  'quantity': 70,
  'symb

In [10]:
data[0]

{'date': 'Sat, 01 Mar 2025 00:00:00 GMT',
 'id': 2,
 'purchase_price': 1600.0,
 'quantity': 10,
 'symbol': 'HDFCBANK.NS'}

In [11]:
P1 = Portfolio()

In [12]:
P1.add_trades(data)

In [13]:
P1

In [14]:
dir(P1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_trade',
 'add_trades',
 'assets_on_dates_oneyear',
 'calculate_current_value',
 'calculate_daily_values_oneyear',
 'calculate_net',
 'get_portfolio_status_by_date',
 'get_portfolio_status_for_dates',
 'net_values',
 'portfolio_status',
 'portfolio_value_for_dates',
 'trades',
 'update_portfolio_status',
 'value_provider']

In [15]:
P1.calculate_net()

({'HDFCBANK.NS': 288500.0,
  'HINDUNILVR.NS': 409000.0,
  'TCS.NS': 285000.0,
  'BBTC.NS': 9000.0,
  'BALKRISIND.NS': 136000.0},
 {'HDFCBANK.NS': 200,
  'HINDUNILVR.NS': 180,
  'TCS.NS': 90,
  'BBTC.NS': 10,
  'BALKRISIND.NS': 100},
 {'HDFCBANK.NS': 1442.5,
  'HINDUNILVR.NS': 2272.222222222222,
  'TCS.NS': 3166.6666666666665,
  'BBTC.NS': 900.0,
  'BALKRISIND.NS': 1360.0},
 {'HDFCBANK.NS': 329960.009765625,
  'HINDUNILVR.NS': 426258.017578125,
  'TCS.NS': 373711.5087890625,
  'BBTC.NS': 21619.49951171875,
  'BALKRISIND.NS': 265219.9951171875})

In [16]:
P1.assets_on_dates_oneyear()

{'2024-03-22': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-23': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-24': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-25': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-26': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-27': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-28': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-29': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-30': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-31': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-04-01': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HIND

In [17]:
P1.calculate_daily_values_oneyear("INR")

([{'date': '2024-03-22', 'assets_quantity': {}},
  {'date': '2024-03-23', 'assets_quantity': {}},
  {'date': '2024-03-24', 'assets_quantity': {}},
  {'date': '2024-03-25', 'assets_quantity': {}},
  {'date': '2024-03-26', 'assets_quantity': {}},
  {'date': '2024-03-27', 'assets_quantity': {}},
  {'date': '2024-03-28', 'assets_quantity': {}},
  {'date': '2024-03-29', 'assets_quantity': {}},
  {'date': '2024-03-30', 'assets_quantity': {}},
  {'date': '2024-03-31', 'assets_quantity': {}},
  {'date': '2024-04-01', 'assets_quantity': {}},
  {'date': '2024-04-02', 'assets_quantity': {}},
  {'date': '2024-04-03', 'assets_quantity': {}},
  {'date': '2024-04-04', 'assets_quantity': {}},
  {'date': '2024-04-05', 'assets_quantity': {}},
  {'date': '2024-04-06', 'assets_quantity': {}},
  {'date': '2024-04-07', 'assets_quantity': {}},
  {'date': '2024-04-08', 'assets_quantity': {}},
  {'date': '2024-04-09', 'assets_quantity': {}},
  {'date': '2024-04-10', 'assets_quantity': {}},
  {'date': '2024-04-

In [18]:
P1.portfolio_value_for_dates(P1.assets_on_dates_oneyear(), "INR")

{}

In [19]:
P1.assets_on_dates_oneyear()

{'2024-03-22': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-23': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-24': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-25': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-26': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-27': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-28': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-29': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-30': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-31': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-04-01': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HIND

In [20]:
P1.portfolio_net()

AttributeError: 'Portfolio' object has no attribute 'portfolio_net'

In [ ]:
P_output = PortfolioOutput(P1, "INR")

In [ ]:
P_output

In [ ]:
P_output.portfolio_net()

({'HDFCBANK.NS': 288500.0,
  'HINDUNILVR.NS': 409000.0,
  'TCS.NS': 285000.0,
  'BBTC.NS': 9000.0,
  'BALKRISIND.NS': 136000.0},
 {'HDFCBANK.NS': 200,
  'HINDUNILVR.NS': 180,
  'TCS.NS': 90,
  'BBTC.NS': 10,
  'BALKRISIND.NS': 100},
 {'HDFCBANK.NS': 1442.5,
  'HINDUNILVR.NS': 2272.222222222222,
  'TCS.NS': 3166.6666666666665,
  'BBTC.NS': 900.0,
  'BALKRISIND.NS': 1360.0},
 {'HDFCBANK.NS': 329960.009765625,
  'HINDUNILVR.NS': 426258.017578125,
  'TCS.NS': 373711.5087890625,
  'BBTC.NS': 21619.49951171875,
  'BALKRISIND.NS': 265219.9951171875})

In [ ]:
P_output.portfolio_daily_value()

{'HDFCBANK.NS': 16498.00048828125,
 'HINDUNILVR.NS': 47362.001953125,
 'TCS.NS': 124570.5029296875,
 'BALKRISIND.NS': 106087.998046875}

In [ ]:
P_output.portfolio_daily_values_oneyear()

{'2024-03-22': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-23': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-24': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-25': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-26': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-27': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-28': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-29': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-30': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-31': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-04-01': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HIND

In [ ]:
P_output.portfolio_value_date_rs()

{}

In [ ]:
output_p = P_output.portfolio_net()

In [ ]:
output_p[1].values()

dict_values([200, 180, 90, 10, 100])

In [ ]:
output_p[0].values()

dict_values([288500.0, 409000.0, 285000.0, 9000.0, 136000.0])

In [ ]:
output_p[2].values()

dict_values([1442.5, 2272.222222222222, 3166.6666666666665, 900.0, 1360.0])

In [ ]:
output_p[3].values()

dict_values([329960.009765625, 426258.017578125, 373711.5087890625, 21619.49951171875, 265219.9951171875])

In [ ]:
P_output.portfolio_value_date_rs()

{}

In [ ]:

class OutputForPHP:
    def __init__(self, currency):
        self.output_list = []
        self.portfolio = Portfolio()
        self.currency = currency

    def output_portfolio(self, trade_data):
        self.portfolio.add_trades(trade_data)
        portfolioOutput = PortfolioOutput(self.portfolio, self.currency)
        output = portfolioOutput.portfolio_net()
        output_currentValue = portfolioOutput.portfolio_daily_value()
        output_dailyvalues2 = portfolioOutput.portfolio_value_date_rs()
        output_dailyvalues = portfolioOutput.portfolio_daily_values_oneyear()

        output_list = [
            [
                key,
                [list(output[1].values())[i], list(output[0].values())[i], list(output[2].values())[i],
                 list(output[3].values())[i]]
            ]
            for i, key in enumerate(output[0].keys())
        ]
        return list(output_dailyvalues2.keys()), list(output_dailyvalues2.values()), output_list, list(output[0].values()), list(output[0].keys()), list(output[3].values())




In [ ]:
P1.calculate_current_value("INR")

{'HDFCBANK.NS': 16498.00048828125,
 'HINDUNILVR.NS': 47362.001953125,
 'TCS.NS': 124570.5029296875,
 'BALKRISIND.NS': 106087.998046875}

In [ ]:
P1.assets_on_dates_oneyear()

{'2024-03-21': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-22': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-23': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-24': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-25': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-26': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-27': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-28': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-29': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-30': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-31': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HIND

In [ ]:
# Output
class PortfolioOutput:
    def __init__(self, portfolio, currency):
        self.portfolio = portfolio
        self.currency = currency

    def portfolio_daily_value(self):
        return self.portfolio.calculate_current_value(self.currency)

    def portfolio_net_cost(self):
        return self.portfolio.calculate_net()[0]

    def portfolio_total_quantity(self):
        return self.portfolio.calculate_net()[1]

    def portfolio_net_cost_per_unit(self):
        return self.portfolio.calculate_net()[2]

    def portfolio_net(self):
        return self.portfolio.calculate_net()

    def portfolio_daily_values_oneyear(self):
        return self.portfolio.assets_on_dates_oneyear()

    def portfolio_value_date_rs(self):
        input_dict = self.portfolio_daily_values_oneyear()
        values = self.portfolio.portfolio_value_for_dates(input_dict, "USD")
        return values

In [24]:
input_dic= P1.assets_on_dates_oneyear()

In [25]:
input_dic

{'2024-03-22': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-23': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-24': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-25': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-26': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-27': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-28': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-29': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-30': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-31': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-04-01': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HIND

In [26]:
P1.portfolio_value_for_dates(input_dic, "INR")

{}

In [34]:
def portfolio_value_for_dates(portfolio_dict, currency):
    stock_data = {}
    value_provider = PortfolioValueProvider()
    startDate = date.today() - timedelta(days=365)
    endDate = date.today()

    for date_str, stocks in portfolio_dict.items():
        date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()  # Convert date string to date object
        for stock_name in stocks.keys():
            if stock_name not in stock_data:
                stock_data[stock_name] = value_provider.get_asset_values(stock_name, "Close", startDate, endDate)

    portfolio_value_by_dates = {}

    for date_str, stocks in portfolio_dict.items():
        # Convert date string to datetime object with time set to midnight
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        date_key = date_obj.strftime('%Y-%m-%d %H:%M:%S')  # Format to match stock_history keys
        total_value = 0

        for stock_name, quantity in stocks.items():
            stock_history = stock_data[stock_name]

            if date_key in stock_history:
                stock_value = stock_history[date_key] * quantity
                total_value += stock_value

        if total_value > 0:
            portfolio_value_by_dates[date_str] = total_value
    return portfolio_value_by_dates

In [35]:
portfolio_value_for_dates(input_dic, "INR")

{'2024-03-26': 865762.0654296875,
 '2024-03-27': 866142.9931640625,
 '2024-03-28': 873699.1442871094,
 '2024-04-01': 884139.599609375,
 '2024-04-02': 884398.7231445312,
 '2024-04-03': 885148.5827636719,
 '2024-04-04': 897089.0649414062,
 '2024-04-05': 899811.3439941406,
 '2024-04-08': 899273.2104492188,
 '2024-04-09': 896992.5903320312,
 '2024-04-10': 896380.7775878906,
 '2024-04-12': 889625.0671386719,
 '2024-04-15': 875046.0327148438,
 '2024-04-16': 877829.3408203125,
 '2024-04-18': 873302.5122070312,
 '2024-04-19': 880499.1003417969,
 '2024-04-22': 881478.7060546875,
 '2024-04-23': 884669.1003417969,
 '2024-04-24': 882495.4040527344,
 '2024-04-25': 878878.6181640625,
 '2024-04-26': 875483.8305664062,
 '2024-04-29': 882692.0422363281,
 '2024-04-30': 878502.6965332031,
 '2024-05-02': 882663.5668945312,
 '2024-05-03': 877435.6787109375,
 '2024-05-06': 888998.2263183594,
 '2024-05-07': 908579.3359375,
 '2024-05-08': 898362.8259277344,
 '2024-05-09': 887695.3979492188,
 '2024-05-10': 891

In [ ]:
input_dic

{'2024-03-22': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-23': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-24': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-25': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-26': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-27': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-28': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-29': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-30': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-03-31': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HINDUNILVR.NS': 160,
  'TCS.NS': 60},
 '2024-04-01': {'BBTC.NS': 10,
  'HDFCBANK.NS': 190,
  'HIND

In [33]:
list(input_dic.keys())

['2024-03-22',
 '2024-03-23',
 '2024-03-24',
 '2024-03-25',
 '2024-03-26',
 '2024-03-27',
 '2024-03-28',
 '2024-03-29',
 '2024-03-30',
 '2024-03-31',
 '2024-04-01',
 '2024-04-02',
 '2024-04-03',
 '2024-04-04',
 '2024-04-05',
 '2024-04-06',
 '2024-04-07',
 '2024-04-08',
 '2024-04-09',
 '2024-04-10',
 '2024-04-11',
 '2024-04-12',
 '2024-04-13',
 '2024-04-14',
 '2024-04-15',
 '2024-04-16',
 '2024-04-17',
 '2024-04-18',
 '2024-04-19',
 '2024-04-20',
 '2024-04-21',
 '2024-04-22',
 '2024-04-23',
 '2024-04-24',
 '2024-04-25',
 '2024-04-26',
 '2024-04-27',
 '2024-04-28',
 '2024-04-29',
 '2024-04-30',
 '2024-05-01',
 '2024-05-02',
 '2024-05-03',
 '2024-05-04',
 '2024-05-05',
 '2024-05-06',
 '2024-05-07',
 '2024-05-08',
 '2024-05-09',
 '2024-05-10',
 '2024-05-11',
 '2024-05-12',
 '2024-05-13',
 '2024-05-14',
 '2024-05-15',
 '2024-05-16',
 '2024-05-17',
 '2024-05-18',
 '2024-05-19',
 '2024-05-20',
 '2024-05-21',
 '2024-05-22',
 '2024-05-23',
 '2024-05-24',
 '2024-05-25',
 '2024-05-26',
 '2024-05-

In [ ]:
list(input_dic.values())

[{'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, 'HINDUNILVR.NS': 160, 'TCS.NS': 60},
 {'BBTC.NS': 10, 'HDFCBANK.NS': 190, '

In [ ]:
startDate

datetime.date(2024, 3, 22)

In [ ]:
endDate

datetime.date(2025, 3, 22)

In [28]:
stock_data_1 = PortfolioValueProvider().get_asset_values("BBTC.NS","Close", startDate, endDate)

In [29]:
len(stock_data_1)

208

In [ ]:
len(list(input_dic.keys()))

366

In [30]:
stock_data_1

{'2024-03-22 00:00:00': 1573.9544677734375,
 '2024-03-26 00:00:00': 1572.1552734375,
 '2024-03-27 00:00:00': 1544.6700439453125,
 '2024-03-28 00:00:00': 1566.908203125,
 '2024-04-01 00:00:00': 1607.986083984375,
 '2024-04-02 00:00:00': 1635.271484375,
 '2024-04-03 00:00:00': 1633.7222900390625,
 '2024-04-04 00:00:00': 1651.412841796875,
 '2024-04-05 00:00:00': 1622.8780517578125,
 '2024-04-08 00:00:00': 1632.4229736328125,
 '2024-04-09 00:00:00': 1597.341796875,
 '2024-04-10 00:00:00': 1590.7952880859375,
 '2024-04-12 00:00:00': 1593.69384765625,
 '2024-04-15 00:00:00': 1550.0672607421875,
 '2024-04-16 00:00:00': 1538.023681640625,
 '2024-04-18 00:00:00': 1516.6351318359375,
 '2024-04-19 00:00:00': 1496.8458251953125,
 '2024-04-22 00:00:00': 1569.456787109375,
 '2024-04-23 00:00:00': 1579.601318359375,
 '2024-04-24 00:00:00': 1582.99951171875,
 '2024-04-25 00:00:00': 1578.05224609375,
 '2024-04-26 00:00:00': 1568.2574462890625,
 '2024-04-29 00:00:00': 1576.453125,
 '2024-04-30 00:00:00

In [ ]:
get_asset_values("HDFCBANK", "India", startDate, endDate)

$HDFCBANK.NS: possibly delisted; no price data found  (1d 2024-03-22 -> 2025-03-22)


[]

In [ ]:
tickerData = yf.Tickers("BBTC.NS")

In [ ]:
yf.__version__

'0.2.55'

In [ ]:
import yfinance as yf